In [ ]:
import os
os.environ["WANDB_DISABLED"]="true"


In [ ]:
! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

In [ ]:
import transformers
print(transformers.__version__)

4.48.3


In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [ ]:
!pip install evaluate

In [ ]:
import evaluate

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("cfilt/iitb-english-hindi")
metric = evaluate.load("sacrebleu")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [ ]:
raw_datasets["train"][0]

{'translation': {'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}

In [ ]:
# Select first 50,000 examples from the train set
small_dataset = raw_datasets["train"].select(range(50000))

# Print sample to verify
print(small_dataset[0])


{'translation': {'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}}


In [ ]:
# ✅ Define train-validation split (90% train, 10% validation)
train_size = int(0.9 * len(small_dataset))  # 90% for training
train_dataset = small_dataset.select(range(train_size))  # First 90% for training
val_dataset = small_dataset.select(range(train_size, len(small_dataset)))  # Last 10% for validation

print(f"Train Dataset: {len(train_dataset)} samples")
print(f"Validation Dataset: {len(val_dataset)} samples")

Train Dataset: 45000 samples
Validation Dataset: 5000 samples


In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'When the banyan 's leaves are still pink and tender, the delicate Map butterfly comes to feast on the sweet sticky smear of syrup on them.', 'hi': 'जब बरगद के नए पत्ते गुलाबी और नर्म होते हैं, तब नाजुक सी मैप तितली उसके मधुर-चिपचिपे रस को चूसने आती है। '}"
1,"{'en': 'First page:', 'hi': 'à¤ªà¥à¤°à¤µà¤¿à¤·à¥à¤à¤¿ à¤à¥à¤¡à¤¼à¥à¤'}"
2,"{'en': 'Whether to show popup notifications when away or busy.', 'hi': 'क्या पॉप अप अधिसूचना दिखानी है जब दूर या व्यस्त है. '}"
3,"{'en': 'The Vrindavan Chandrodaya Mandir will strive to disseminate the philosophical message of the Bhagavad Gita and Srimad Bhagavatam in an easy to understand and rational manner –that message is particularly relevant for us as a society today.', 'hi': 'वृंदावन चंद्रोदय मंदिर सरल और तार्किक विधि से भगवद् गीता और श्रीमद् भागवतम् के दार्शनिक संदेश का प्रसार करने का प्रयत्न करेगा। वर्तमान समाज के रूप में यह संदेश हमारे लिए विशेष रूप से प्रासंगिक है।'}"
4,"{'en': 'are 100 percent.', 'hi': 'पूरी सौ प्रतिशत। '}"


In [ ]:
metric

EvaluationModule(name: "sacrebleu", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'e

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[12110, 2, 90, 131, 8800, 61, 0], [239, 23, 414, 8800, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[2204, 10967, 818, 2, 90, 44, 4625, 44, 16, 4072, 1936, 5386, 61, 0], [44, 1687, 23, 44, 980, 9972, 581, 44, 16, 4072, 1936, 5386, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "hi"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(small_dataset[:2])

{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0]]}

In [ ]:
# ✅ Tokenize train and validation sets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, num_proc=4)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True, num_proc=4)

print(f"Tokenized Train Dataset: {len(tokenized_train_dataset)} samples")
print(f"Tokenized Validation Dataset: {len(tokenized_val_dataset)} samples")

Map (num_proc=4):   0%|          | 0/45000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Tokenized Train Dataset: 45000 samples
Tokenized Validation Dataset: 5000 samples


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,  # Use tokenized train dataset
    eval_dataset=tokenized_val_dataset,  # Use tokenized validation dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [ ]:
tokenized_datasets = small_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-53-cc641cd089b7>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.290500,0.685730,79.190000,8.131600


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=5625, training_loss=0.37596414523654514, metrics={'train_runtime': 903.7443, 'train_samples_per_second': 49.793, 'train_steps_per_second': 6.224, 'total_flos': 178466416754688.0, 'train_loss': 0.37596414523654514, 'epoch': 1.0})

Manually Test Model Translations

In [ ]:
sample_text = "I feel good"  # English sentence

# Tokenize input
inputs = tokenizer(sample_text, return_tensors="pt").to("cuda")  # Move to GPU if available

# Generate translation
outputs = model.generate(**inputs)
translated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print("English:", sample_text)
print("Hindi Translation:", translated_text[0])


English: I feel good
Hindi Translation: मैं अच्छा महसूस करता हूँ


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load original model (before fine-tuning)
baseline_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
baseline_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

# Translate sample text with baseline model
inputs = baseline_tokenizer(sample_text, return_tensors="pt")
outputs = baseline_model.generate(**inputs)
baseline_translation = baseline_tokenizer.batch_decode(outputs, skip_special_tokens=True)

print("Baseline Translation:", baseline_translation[0])
print("Fine-Tuned Model Translation:", translated_text[0])


Baseline Translation: मैं अच्छा लग रहा है
Fine-Tuned Model Translation: मैं अच्छा महसूस करता हूँ


In [ ]:
ls -lh

total 12K
drwx------ 6 root root 4.0K Feb 18 06:57 drive/
drwxr-xr-x 6 root root 4.0K Feb 18 07:45 opus-mt-en-hi-finetuned-en-to-hi/
drwxr-xr-x 1 root root 4.0K Feb 13 14:19 sample_data/


In [ ]:
ls -lh opus-mt-en-hi-finetuned-en-to-hi/


total 16K
drwxr-xr-x 2 root root 4.0K Feb 18 07:44 checkpoint-5000/
drwxr-xr-x 2 root root 4.0K Feb 18 07:45 checkpoint-5500/
drwxr-xr-x 2 root root 4.0K Feb 18 07:45 checkpoint-5625/
drwxr-xr-x 5 root root 4.0K Feb 18 07:35 runs/


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint_path = "./opus-mt-en-hi-finetuned-en-to-hi/checkpoint-5625"

#  Load model and tokenizer from last checkpoint
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

print(" Model reloaded successfully from last checkpoint!")


 Model reloaded successfully from last checkpoint!


In [ ]:
save_path = "./hindi_translation_model"

#  Save model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(" Model saved successfully at:", save_path)


 Model saved successfully at: ./hindi_translation_model


In [ ]:
ls -lh ./hindi_translation_model/


total 294M
-rw-r--r-- 1 root root 1.4K Feb 18 09:09 config.json
-rw-r--r-- 1 root root  288 Feb 18 09:09 generation_config.json
-rw-r--r-- 1 root root 290M Feb 18 09:09 model.safetensors
-rw-r--r-- 1 root root 794K Feb 18 09:09 source.spm
-rw-r--r-- 1 root root  416 Feb 18 09:09 special_tokens_map.json
-rw-r--r-- 1 root root 1.1M Feb 18 09:09 target.spm
-rw-r--r-- 1 root root  849 Feb 18 09:09 tokenizer_config.json
-rw-r--r-- 1 root root 2.2M Feb 18 09:09 vocab.json


In [ ]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

In [ ]:
from getpass import getpass
import os

# Securely enter Hugging Face API Token (input is hidden)
hf_token = getpass("Enter your Hugging Face API Token: ")

# Login using the token
os.system(f"huggingface-cli login --token {hf_token}")


Enter your Hugging Face API Token: ··········


0

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
user_info = api.whoami()
hf_username = user_info["name"]  #  Get the correct username

print("Your Hugging Face Username:", hf_username)


Your Hugging Face Username: rairashmi


In [ ]:
repo_name = "english2hindi-translation-model"  #  Keep this unique
repo_id = f"{hf_username}/{repo_name}"

# Create the repo (if it doesn’t exist)
api.create_repo(repo_id, exist_ok=True)

print(f" Repository created: https://huggingface.co/{repo_id}")


✅ Repository created: https://huggingface.co/rairashmi/english2hindi-translation-model


In [ ]:
ls -lh ./hindi_translation_model/


total 294M
-rw-r--r-- 1 root root 1.4K Feb 18 09:09 config.json
-rw-r--r-- 1 root root  288 Feb 18 09:09 generation_config.json
-rw-r--r-- 1 root root 290M Feb 18 09:09 model.safetensors
-rw-r--r-- 1 root root 794K Feb 18 09:09 source.spm
-rw-r--r-- 1 root root  416 Feb 18 09:09 special_tokens_map.json
-rw-r--r-- 1 root root 1.1M Feb 18 09:09 target.spm
-rw-r--r-- 1 root root  849 Feb 18 09:09 tokenizer_config.json
-rw-r--r-- 1 root root 2.2M Feb 18 09:09 vocab.json


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

repo_id = "rairashmi/english2hindi-translation-model"  # Your Hugging Face repo ID

# Load the saved model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("./hindi_translation_model")
tokenizer = AutoTokenizer.from_pretrained("./hindi_translation_model")

# Push model & tokenizer to Hugging Face Hub
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

print(f" Model uploaded successfully: https://huggingface.co/{repo_id}")


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


model.safetensors:   0%|          | 0.00/304M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

 Model uploaded successfully: https://huggingface.co/rairashmi/english2hindi-translation-model


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

repo_id = "rairashmi/english2hindi-translation-model"

# Load model & tokenizer from Hugging Face Hub
model = AutoModelForSeq2SeqLM.from_pretrained(repo_id)
tokenizer = AutoTokenizer.from_pretrained(repo_id)

# Test translation
sample_text = "Hello, how are you?"
inputs = tokenizer(sample_text, return_tensors="pt")
outputs = model.generate(**inputs)
translated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print("Translated Output:", translated_text[0])


config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/304M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/849 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


✅ Translated Output: हैलो, तुम कैसे हो?
